# 1. Definition of ```biobeam```-simulation

Deterministic parameters for the simulation

In [4]:
import os
import json
import numpy as np

param_save_path = '/scratch/hoerl/small_spheroid_3/params1.json'
params = {}

### save path formatstring for simulated images 
# has to contain the following placeholders:
# {x} := index of x tile
# {y} := index of y tile
# {z} := index of z tile
# {illum} := 1=left/2=right
# {lam} := wavelength
save_fstring = params['save_fstring'] = '/scratch/hoerl/small_spheroid_3/sim-bbeam1-x{x}-y{y}-z{z}-i{illum}-l{lam}-view{view}/'

seed = params['seed'] = 1222
raw_data_path = params['raw_data_path'] = '/scratch/hoerl/small_spheroid_3/sim-phantom/'
dn_template_path = params['dn_template_path'] = '/scratch/hoerl/small_spheroid_3/sim-dn/'
raw_data_dims = params['raw_data_dims'] = (512, 1024, 1024)

# NB: only use downsampling for quick, local testing, do final sims on raw resolution!
downsampling = params['downsampling'] = 2


# NB: z should not be subsampled so much (psz should be ~lambda/2)
# TODO: re-do with larger volume
#phys_dims = params['phys_dims'] = (400, 400, 50)
phys_dims = params['phys_dims'] = (200, 200, 100)

na_illum = params['na_illum'] = 0.15
na_detect = params['na_detect'] = 0.5
ri_medium =  params['ri_medium'] = 1.33
ri_delta_range = params['ri_delta_range'] = (.01, .01)
clip_max_ri = params['clip_max_ri'] = 1.0
two_sided_illum = params['two_sided_illum'] = True
lambdas = params['lambdas'] = (500, )

# whether to simulate all images again with specimen rotated 180deg around y axis
two_views = params['two_views'] = True

fov_size = params['fov_size'] = (200, 360, 360)
# smaller fov for points so we have no points in overlap
point_fov_size = params['point_fov_size'] = (120, 150, 150)
n_points_per_fov = params['n_points_per_fov'] = 16
points_min_distance = params['points_min_distance'] = 20

min_off = params['min_off'] = (-20, -20, -20)
max_off = params['max_off'] = (20, 20, 20)

#x_locs = params['x_locs'] = (1024 - 560, 1024 - 180, 1024 + 180, 1024 + 560)
x_locs = params['x_locs'] = (512 - 100, 512 + 100)
#x_locs = params['x_locs'] = (1024,)
#y_locs = params['y_locs'] = (1024 - 560, 1024 - 180, 1024 + 180, 1024 + 560)
#y_locs = params['y_locs'] = (1024 - 600, )
y_locs = params['y_locs'] = (512, )

z_locs = params['z_locs'] = (512//2, )

# maybe set this to 32?
padding = params['padding'] = 16
# increase these if GPU memory is not enough
conv_subblocks = params['conv_subblocks'] = (1,1)
bprop_nvolumes = params['bprop_nvolumes'] = 1

# set to True to only simulate the planes necessary to cut all tiles specified
# NB: we can not arbitraily re-cut fovs from simulated data in that case
only_necessary_planes = params['only_necessary_planes'] = True

# set this != None to simulate a grid of point sources with given spacing
# instead of random spots
grid_descs = params['grid_descs'] = 20


#os.path.relpath(raw_data_path, os.path.split(param_save_path)[0])
#params
#raw_data_path

# 2. Random offsets and descriptors

Generating random offsets of tiles and descriptors in each tile

In [5]:
from sim_util import random_points_in_volume_min_distance

np.random.seed(seed)
fields = params['fields'] = {}

xidx, yidx, zidx = np.meshgrid(range(len(x_locs)), range(len(y_locs)), range(len(z_locs)))
for xi, yi, zi in zip(xidx.flat, yidx.flat, zidx.flat):
    #print(xi, yi, zi)
    
    off = [np.random.randint(min_off[i], max_off[i] + 1) for i in range(len(min_off))]
    loc = [z_locs[zi], y_locs[yi], x_locs[xi]]
    min_ = np.array(loc) - np.ceil(np.array(fov_size) / 2)
    max_ = np.array(loc) + np.floor(np.array(fov_size) / 2)
    
    min_points = min_ + (np.array(fov_size) - np.array(point_fov_size)) / 2
    max_points = min_ + (np.array(fov_size) + np.array(point_fov_size)) / 2
    
    points = random_points_in_volume_min_distance(min_points, max_points, points_min_distance, n_points_per_fov)
    
    fields[','.join(map(str, (xi, yi, zi)))] = {}
    fields[','.join(map(str, (xi, yi, zi)))]['off'] = off
    fields[','.join(map(str, (xi, yi, zi)))]['points'] = [list(map(int, p)) for p in list(points)]
    

# Save simulation params

In [6]:
with open(param_save_path, 'w') as fd:
    json.dump(params, fd, indent=1)

# Run simulation, better to do this via run_sim2.py

In [ ]:
import gputools
import pyopencl

# set and check which gpu is used
gputools.init_device(id_platform = 0, id_device = 0)
gputools.get_device()

In [ ]:
from sim_util import sim_from_definition

param_save_path = '/Volumes/davidh-ssd/mv-sim/sim_tissue/bbeam_params.json'
sim_from_definition(param_save_path)

In [ ]:
from sim_util import random_points_in_volume_min_distance

# Misc. stuff

In [ ]:
import numpy as np

img = np.zeros([12,13,14])
np.meshgrid(*[np.arange(0,n) for n in img.shape])[0].shape

In [ ]:
# Approximately Normalize and scale to desired SNR
# TODO: extract to function

img = load_tiff_sequence('/Volumes/davidh-ssd/mv-sim/sim_tissue/sim-bbeam-x0-y0-z0-iright-l400/', 'bbeam_sim_c1.*')

snr = 7
q = 0.98
img = img / np.quantile(img, q) * snr**2

save_as_sequence(img, '/Volumes/davidh-ssd/mv-sim/sim_tissue/sim-bbeam-x0-y0-z0-iright-l400/', file_pattern='bbeam_sim_c1_z{plane}.tif')


In [ ]:
from functools import reduce
from operator import add
from matplotlib import pyplot as plt
from scipy.spatial import KDTree
%matplotlib inline

# plot points
gt = reduce(add, [v['points'] for v in fields.values()])
plt.plot([g[2] for g in gt], [g[1] for g in gt], '.')

# plot nn-distance hist
kd = KDTree(gt)
nndist = [kd.query(g, 2)[0][1] for g in gt]
plt.figure()
plt.hist(nndist)

In [ ]:
two_sided_illum = True
from itertools import product

for lam, right_illum in product(lambdas, ([False] if not two_sided_illum else [True, False])):
    print(lam, right_illum)